In [22]:
import pandas as pd
import os
import plotly.express as px
dados_2024 = pd.read_csv('dados/2024_Viagem.csv', sep=';', encoding='latin-1')
dados_2024.head(5)

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,...,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos
0,18831091,000011/24,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26425,Instituto Federal do Acre,***.405.257-**,...,-1,Sem informação,23/02/2024,25/02/2024,Rio Branco/AC,Ministrar a disciplina de Conceitos e Aplicaçõ...,"929,18","0,00","0,00","0,00"
1,18831495,000001/24,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26274,Fundação Universidade Federal de Uberlândia,***.587.016-**,...,-1,Sem informação,18/01/2024,22/01/2024,São Paulo/SP,Participação nas atividades do ESTAGIO DE CAPA...,"0,00","0,00","0,00","0,00"
2,18831777,000002/24,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26274,Fundação Universidade Federal de Uberlândia,***.587.016-**,...,-1,Sem informação,29/02/2024,04/03/2024,São Paulo/SP,Participação nas atividades do ESTAGIO DE CAPA...,"0,00","0,00","0,00","0,00"
3,18831821,000003/24,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26274,Fundação Universidade Federal de Uberlândia,***.587.016-**,...,-1,Sem informação,11/04/2024,15/04/2024,São Paulo/SP,Participação nas atividades do ESTAGIO DE CAPA...,"0,00","0,00","0,00","0,00"
4,19177818,000001/24-1C,Realizada,SIM,Inclusão das diárias.,26000,Ministério da Educação,26271,Fundação Universidade de Brasília,***.626.601-**,...,-1,Sem informação,29/01/2024,06/02/2024,"Lille/França, Paris/França",Programação dos eventos que a Professora irá p...,"14176,38","6892,31","0,00","0,00"


Analisando os dados e descobrindo a quantidade de gastos por ministério em 2024. Vale destacar que é possível obter todos os mesmos valores com esses códigos para os anos anteriores, mas por conta do volume de dados e tempo de processamento necessário para executar o código, optamos por colocar somente  a tabela de 2024. As demais tabelas estão disponíveis no site https://portaldatransparencia.gov.br/download-de-dados/viagens . 

In [20]:
dados_2024['Total gasto'] = dados_2024[['Valor diárias', 'Valor passagens', 'Valor outros gastos']].apply(lambda row: row.str.replace(',', '.').astype(float).sum(), axis=1)
dados_realizadas_2024 = dados_2024[dados_2024['Situação'] == 'Realizada']
gastos_por_orgao_2024 = dados_realizadas_2024.groupby('Nome do órgão superior')['Total gasto'].sum()
gastos_por_orgao_2024.sort_values(ascending=False)



Nome do órgão superior
Sem informação                                                   6.857747e+08
Ministério da Justiça e Segurança Pública                        3.214306e+08
Ministério da Defesa                                             3.047604e+08
Ministério da Educação                                           2.681042e+08
Ministério do Meio Ambiente e Mudança do Clima                   1.134884e+08
Ministério da Saúde                                              8.452008e+07
Ministério das Relações Exteriores                               7.438678e+07
Ministério da Previdência Social                                 6.694531e+07
Ministério do Planejamento e Orçamento                           5.295148e+07
Ministério dos Transportes                                       4.848881e+07
Presidência da República                                         4.513057e+07
Ministério da Agricultura e Pecuária                             4.037960e+07
Ministério do Desenvolvimento Agrário e A

In [21]:
gastos_por_orgao_2024.to_csv('gastos_por_orgao.csv', header=True, encoding='latin-1')

In [4]:
total_gasto_2024 = dados_realizadas_2024['Total gasto'].sum()
gastos_sem_informacao_2024 = dados_realizadas_2024.loc[dados_realizadas_2024['Nome do órgão superior'] == 'Sem informação', 'Total gasto'].sum()
porcentagem_gastos_sem_informacao_2024 = (gastos_sem_informacao_2024 / total_gasto_2024) * 100
porcentagem_gastos_sem_informacao_2024 

30.780998594868812

Com o código acima, foi possível descobrir a porcentagem de gastos em viagens sigilosas (Sem informação). Agora vou fazer um dataframe para a presidência com o objetivo de descobrir os países que mais receberam viagens.

In [7]:
df_presidencia = dados_realizadas_2024[dados_realizadas_2024['Nome do órgão superior'] == 'Presidência da República'].copy()

def classificar_viagem(destino):
    if destino == "Informações protegidas por sigilo" or destino == "Sem informação":
        return 'Não informada'
    estados_brasil = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
    
    if any(estado in destino for estado in estados_brasil):
        return 'Nacional'
    else:
        return 'Internacional'

df_presidencia.loc[:, 'Tipo de Viagem'] = df_presidencia['Destinos'].apply(classificar_viagem)

df_internacionais = df_presidencia[df_presidencia['Tipo de Viagem'] == 'Internacional'].copy()

df_internacionais['País'] = df_internacionais['Destinos'].apply(lambda x: x.split('/')[-1].strip() if '/' in x else x)

top_paises = df_internacionais['País'].value_counts()

top_paises.to_csv('top_paises.csv', header=True)


No código acima, eu descobri qual viagens são internacionais e quais são nacionais, e gerei uma tabela com os principais destinos internacionais. Agora vou descobrir os paíes que tiveram mais gastos com viagens.

In [8]:
df_internacionais = df_presidencia[df_presidencia['Tipo de Viagem'] == 'Internacional'].copy()

df_internacionais['País'] = df_internacionais['Destinos'].apply(lambda x: x.split('/')[-1].strip() if '/' in x else x)

gastos_por_pais = df_internacionais.groupby('País')['Total gasto'].sum()

top_gastos_pais = gastos_por_pais.sort_values(ascending=False)

top_gastos_pais

top_gastos_pais.to_csv('top_gastos_pais.csv', header=True)



In [9]:
dados_2024['Período - Data de início'] = pd.to_datetime(dados_2024['Período - Data de início'], format='%d/%m/%Y')
dados_2024['Mês'] = dados_2024['Período - Data de início'].dt.month

dados_realizadas_2024 = dados_2024[dados_2024['Situação'] == 'Realizada']
gastos_mensais_totais = dados_realizadas_2024.groupby('Mês')['Total gasto'].sum()

gastos_mensais_totais = gastos_mensais_totais.reindex([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

gastos_mensais_totais.index = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

gastos_mensais_totais.to_csv('Gastos mês a mês.csv', header=True, encoding='latin-1')
gastos_mensais_totais.head()

Janeiro      2.406410e+08
Fevereiro    1.273171e+08
Março        1.751891e+08
Abril        1.787477e+08
Maio         1.863196e+08
Name: Total gasto, dtype: float64

O código acima me permitiu descobrir os gastos mês a mês do governo federal.

E agora para finalizar, vou descobrir quais foram as 10 viagens mais caras do governo federal e da presidência. 

In [14]:
top_10_viagens = dados_realizadas_2024.nlargest(10, 'Total gasto')

top_10_viagens.to_csv('top_10_viagens.csv', index=False, encoding='latin-1')

viagens_presidencia = dados_realizadas_2024[dados_realizadas_2024['Nome do órgão superior'] == 'Presidência da República']
top_10_viagens_presidencia = viagens_presidencia.nlargest(10, 'Total gasto')
top_10_viagens_presidencia.to_csv('top_10_viagens_presidencia.csv', index=False, encoding='latin-1')

In [18]:
viagens_mauro_vieira = dados_realizadas_2024[dados_realizadas_2024['Nome'] == 'MAURO LUIZ IECKER VIEIRA']
viagens_mauro_vieira.to_csv('viagens_mauro_vieira.csv', index=False, encoding='latin-1')